In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skewnorm
from scipy.optimize import minimize_scalar
from scipy.stats import skewnorm
from bayes_opt import BayesianOptimization
from simulation import Simulation
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from hyperopt import fmin, tpe, Trials, space_eval, STATUS_OK, hp

In [2]:
data = pd.read_csv('cluster_real_distributions.csv').drop(['Unnamed: 0', 'image', 'condition'], axis=1)
write_path = "outputs_7"

In [3]:
def scaled_gaussian(x, mu, sigma):
    max_val = 1 / (sigma * np.sqrt(2 * np.pi))
    pdf_val = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    y = pdf_val / max_val
    return y

class ScaledSkewNormal:

    def __init__(self, a, mu, sigma):
        neg_pdf = lambda x: -skewnorm.pdf(x, a, mu, sigma)
        self.result = minimize_scalar(neg_pdf, bounds=(mu - 3*sigma, mu + 3*sigma), method='bounded')
        self.pdf_max_value = skewnorm.pdf(self.result.x, a, mu, sigma)
        self.a = a
        self.mu = mu
        self.sigma = sigma

    def get_value(self, x):
        pdf_val = skewnorm.pdf(x, self.a, self.mu, self.sigma)
        return pdf_val / self.pdf_max_value

def time_dist(x, k=0.2):
    return (1 - np.exp(-k*x))

In [4]:
def derive_target(
    mean_separation: float,
    volume: float,
    sphericity: float,
    n_cells: float,
    max_time: float
) -> float:
    """
    Derive the value to be maximised from the results of the simulation.
    
    Inputs:
        results: The results of the simulation.
    
    Returns:
        The value to be maximised.
    """

    skew_normal_volume = ScaledSkewNormal(5, 80, 100)
    skew_normal_n_cells = ScaledSkewNormal(5, 60, 120)
    skew_normal_sphericity = ScaledSkewNormal(-5, 0.97, 0.1)

    mean_separation_optimisation_value = scaled_gaussian(mean_separation, mu=-0.1, sigma=0.2)
    volume_optimisation_value = skew_normal_n_cells.get_value(volume)
    sphericity_optimisation_value = skew_normal_sphericity.get_value(sphericity)
    n_cells_optimisation_value = skew_normal_volume.get_value(n_cells)
    time_optimisation_value = time_dist(max_time)

    target_value = (
        mean_separation_optimisation_value *
        volume_optimisation_value *
        sphericity_optimisation_value *
        n_cells_optimisation_value *
        time_optimisation_value
    )

    return target_value

def target_function(
    params: dict
) -> float:
    """
    """

    simulation = Simulation(N_bodies=10)
    try:
        simulation.execute(
            alpha=params['alpha'],
            beta=params['beta'],
            A_eq_star_scaling=params['A_eq_star_scaling'],
            P_star=params['P_star'],
            radius_scaling=params['radius_scaling'],
        )
        
        results = simulation.results.iloc[-1]
        target = -derive_target(
            results['mean_separation'],
            results['cluster_vol'],
            results['sphericity'],
            results['final_N_bodies'],
            results['t']
        )
    except:
        target = 1

    return {'loss': target, 'status': STATUS_OK}

In [7]:
search_space = {
    'alpha': hp.uniform('alpha', 0, 10_000),
    'beta': hp.uniform('beta', 0, 10_000),
    'A_eq_star_scaling': hp.uniform('A_eq_star_scaling', 0, 1),
    'P_star': hp.uniform('P_star', 0, 10_000),
    'radius_scaling': hp.uniform('radius_scaling', 0.6, 1.8),
}

In [9]:
trials = Trials()

algo = tpe.suggest
max_evals = 10

# Perform the optimization
best = fmin(
    fn=target_function,
    space=search_space,
    algo=algo,
    max_evals=max_evals,
    trials=trials
)

100%|██████████| 10/10 [00:00<00:00, 112.66trial/s, best loss: 1.0]


In [ ]:
trials.

{'A_eq_star_scaling': 0.6951280930422716,
 'P_star': 398.4789589920745,
 'alpha': 4273.047989664059,
 'beta': 821.7080250908393,
 'radius_scaling': 0.9349292834999221}

In [18]:
logger = JSONLogger(path="./logs_2.log")

search_space = {
    'alpha': (0, 10_000),
    'beta': (0, 10_000),
    'A_eq_star_scaling': (0, 1),
    'P_star': (0, 10_000),
    'radius_scaling': (0.6, 1.8),
}

optimizer = BayesianOptimization(
    f=target_function,
    pbounds=search_space,
    verbose=2
)

optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

optimizer.maximize(
    init_points=10,
    n_iter=10_000_000,
)

KeyboardInterrupt: 

In [17]:
len(optimizer.res)

50002